to run presentation, enter `jupyter nbconvert Tutorial.ipynb --to slides --post serve` in the terminal

# Text Classification with BERT

<img src="img/bert_reading.jpg" style="width: 500px;"/>

In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

In [2]:
import os
import re

import pandas as pd
pd.set_option('display.max_colwidth', 0)

import numpy as np
np.random.seed(123)

import tensorflow as tf
from tensorflow import keras

from tqdm import tqdm

from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

import datetime
import math

BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

## Workshop Goal in Machine Learning Jargon:

_Leverage a **bidirectional language model** with a **transformer architecture** for **text classification** using **transfer learning**_

Right now, this should make no sense!

But by the end of this workshop, you will not only understand what this means, but you will be able to use this method to train a natural language processing model with only 500 data points.

**Note:** This code in this tutorial is largely based on Google's tutorial available [here](https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb) and this Towards Data Science post available [here](https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b).

# The Road Ahead

**Goal:** _Leverage a **bidirectional language model** with a **transformer architecture** for **text classification** using **transfer learning**_

1. Text Classification
2. Transfer Learning
3. Bidirectional Language Model
4. Transformer Architecture
5. Put it all together!

# 1. Text Classification

Text classification is... classifying text.

### Sentiment analysis:
<img src="img/harvard_review.png" style="width: 500px;"/>

### Topic classification
<img src="img/topic_classification.png" style="width: 500px;"/>

## IMDb Sentiment Analysis

Today, we're going to be working with the IMDb sentiment analysis dataset.

**The Task:** Classify a movie review on IMDb as <span style="color:green">positive</span> or <span style="color:red">negative</span>.

## Loading the Data

We will be using the IMDB Large Movie Review Dataset, which is hosted by Stanford.

**TODO: Look into hosting data ourselves**

In [0]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in tqdm(os.listdir(directory), desc=os.path.basename(directory)):
        with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
    train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))

    return train_df, test_df

We now define a class that will handle downloading and preparing the text

```python
class MovieReviewData:
    DATA_COLUMN = "sentence"
    LABEL_COLUMN = "polarity"

    def __init__(self, tokenizer: FullTokenizer, sample_size=None, max_seq_len=128):
        # Download the data
        self.tokenizer = tokenizer
        self.sample_size = sample_size
        self.max_seq_len = 0
        train, test = download_and_load_datasets()
        
        # Re-index by length of text
        train, test = map(lambda df: 
                          df.reindex(df[MovieReviewData.DATA_COLUMN].str.len().sort_values().index), 
                          [train, test])
        
        # sample the data
        if sample_size is not None:
            assert sample_size % 128 == 0
            train, test = train.head(sample_size), test.head(sample_size)
            # train, test = map(lambda df: df.sample(sample_size), [train, test])
        
        # prepare the data
        ((self.train_x, self.train_y),
         (self.test_x, self.test_y)) = map(self._prepare, [train, test])
        
        # pad the data as needed
        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        ((self.train_x, self.train_x_token_types),
         (self.test_x, self.test_x_token_types)) = map(self._pad, 
                                                       [self.train_x, self.test_x])
```

```python
def _prepare(self, df):
    x, y = [], []
    with tqdm(total=df.shape[0], unit_scale=True) as pbar:
        for ndx, row in df.iterrows():
            # get text and label from row of data
            text, label = row[MovieReviewData.DATA_COLUMN], row[MovieReviewData.LABEL_COLUMN]

            # tokenize
            tokens = self.tokenizer.tokenize(text)

            # append special start and end tokens
            tokens = ["[CLS]"] + tokens + ["[SEP]"]

            # convert tokens to IDs
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)

            # clip texts to keep it under the max sequence length threshold
            self.max_seq_len = max(self.max_seq_len, len(token_ids))

            # save results
            x.append(token_ids)
            y.append(int(label))
            pbar.update()
    return np.array(x), np.array(y)
```

```python
def _pad(self, ids):
        # The function pads the texts if they are less than the max sequence length threshold
        x, t = [], []
        token_type_ids = [0] * self.max_seq_len
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
            t.append(token_type_ids)
        return np.array(x), np.array(t)
```

In [0]:
class MovieReviewData:
    DATA_COLUMN = "sentence"
    LABEL_COLUMN = "polarity"

    def __init__(self, tokenizer, sample_size=None, max_seq_len=128):
        # Download the data
        self.tokenizer = tokenizer
        self.sample_size = sample_size
        self.max_seq_len = 0
        train, test = download_and_load_datasets()
        
        # Re-index by length of text
        train, test = map(lambda df: df.reindex(df[MovieReviewData.DATA_COLUMN].str.len().sort_values().index), 
                          [train, test])
        
        # sample the data
        if sample_size is not None:
            assert sample_size % 128 == 0
            self.train, self.test = train.sample(sample_size), test.sample(sample_size)
            # train, test = map(lambda df: df.sample(sample_size), [train, test])
        
        # prepare the data
        ((self.train_x, self.train_y),
         (self.test_x, self.test_y)) = map(self._prepare, [self.train, self.test])
        
        # pad the data as needed
        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        ((self.train_x, self.train_x_token_types),
         (self.test_x, self.test_x_token_types)) = map(self._pad, 
                                                       [self.train_x, self.test_x])

    def _prepare(self, df):
        x, y = [], []
        with tqdm(total=df.shape[0], unit_scale=True) as pbar:
            for ndx, row in df.iterrows():
                # get text and label from row of data
                text, label = row[MovieReviewData.DATA_COLUMN], row[MovieReviewData.LABEL_COLUMN]
                
                # tokenize
                tokens = self.tokenizer.tokenize(text)
                
                # append special start and end tokens
                tokens = ["[CLS]"] + tokens + ["[SEP]"]
                
                # convert tokens to IDs
                token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
                
                # clip texts to keep it under the max sequence length threshold
                self.max_seq_len = max(self.max_seq_len, len(token_ids))
                
                # save results
                x.append(token_ids)
                y.append(int(label))
                pbar.update()
        return np.array(x), np.array(y)

    def _pad(self, ids):
        # The function pads the texts if they are less than the max sequence length threshold
        x, t = [], []
        token_type_ids = [0] * self.max_seq_len
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
            t.append(token_type_ids)
        return np.array(x), np.array(t)

## We'll come back to this later...

**Goal:** _Leverage a **bidirectional language model** with a **transformer architecture** for <span style="color:green;font-weight: bold">text classification</span> using **transfer learning**_

# 2. Transfer Learning

This was yesterday's topic so we won't go into too much detail.

Transfer learning "focuses on storing knowledge gained while solving one problem and applying it to a different but related problem" (Wikipedia).

<div style="display: flex">
    <img src="img/math.jpeg" style="width: 400px;"/>
    <img src="img/cs.jpeg" style="width: 400px;"/>
</div>

**Goal:** _Leverage a **bidirectional language model** with a **transformer architecture** for <span style="color:green;font-weight: bold">text classification</span> using <span style="color:green;font-weight: bold">transfer learning</span>_

# 3. Bidirectional Language Model

Bidirectional language modelling will be our "first task" in the transfer learning framework. 

We will transfer what we learn from bidirectional language modelling to the IMDb sentiment analysis task.

## But What Is Bidirectional Language Modelling?

## But What Is ~~Bidirectional~~ Language Modelling?


<img src="img/LM.jpeg"/>


## But What Is Bidirectional Language Modelling?

<img src="img/BiLM.png"/>


This task is self-supervised, which means it can be scaled to enormous amounts of data!

**Goal:** _Leverage a <span style="color:green;font-weight: bold">bidirectional language model</span> with a **transformer architecture** for <span style="color:green;font-weight:bold">text classification</span> using <span style="color:green;font-weight: bold">transfer learning</span>_

# 4. Transformer Architecture

Before you can understand transformers, you need to understand attention.

## Dot Product Attention
<img src="img/attention.png"/>
https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3

## Dot Product Attention

Can capture long-range dependencies better than recurrent models.

## Transformers
<img src="img/attention_all.png" style="width:600px"/>

## Transformers
<img src="img/transformer.png" style="width:300px"/>
Improving Language Understanding by Generative Pre-Training

## Self Attention
It's almost attention, but everything is in terms of the input tokens.

<img src="img/attention.png"/>


# BERT

BERT basically just uses a transformer architecture for bidirectional language modelling.

Trained on over 3 billion words!

Google has released the model on tensorflow-hub.

To transfer BERT to sentence level tasks like sentiment analysis, we represent the sentence with the encoding for the class token.

<img src="img/BiLM.png"/>


**Goal:** _Leverage a <span style="color:green;font-weight: bold">bidirectional language model</span> with a <span style="color:green;font-weight: bold">transformer architecture</span> for <span style="color:green;font-weight:bold">text classification</span> using <span style="color:green;font-weight: bold">transfer learning</span>_

Hurray! Now we can code it up.

# 5. Put it all together!

## Load BERT

In [0]:
bert_ckpt_dir="gs://bert_models/2018_10_18/uncased_L-12_H-768_A-12/"
bert_ckpt_file = bert_ckpt_dir + "bert_model.ckpt"
bert_config_file = bert_ckpt_dir + "bert_config.json"

In [6]:
%%time

bert_model_dir="2018_10_18"
bert_model_name="uncased_L-12_H-768_A-12"

!mkdir -p .model .model/$bert_model_name

for fname in ["bert_config.json", "vocab.txt", "bert_model.ckpt.meta", "bert_model.ckpt.index", "bert_model.ckpt.data-00000-of-00001"]:
  cmd = f"gsutil cp gs://bert_models/{bert_model_dir}/{bert_model_name}/{fname} .model/{bert_model_name}"
  !$cmd

!ls -la .model .model/$bert_model_name

Copying gs://bert_models/2018_10_18/uncased_L-12_H-768_A-12/bert_config.json...
/ [1 files][  313.0 B/  313.0 B]                                                
Operation completed over 1 objects/313.0 B.                                      
Copying gs://bert_models/2018_10_18/uncased_L-12_H-768_A-12/vocab.txt...
/ [1 files][226.1 KiB/226.1 KiB]                                                
Operation completed over 1 objects/226.1 KiB.                                    
Copying gs://bert_models/2018_10_18/uncased_L-12_H-768_A-12/bert_model.ckpt.meta...
/ [1 files][883.1 KiB/883.1 KiB]                                                
Operation completed over 1 objects/883.1 KiB.                                    
Copying gs://bert_models/2018_10_18/uncased_L-12_H-768_A-12/bert_model.ckpt.index...
/ [1 files][  8.3 KiB/  8.3 KiB]                                                
Operation completed over 1 objects/8.3 KiB.                                      
Copying gs://bert_models/2

In [0]:
bert_ckpt_dir    = os.path.join(".model/",bert_model_name)
bert_ckpt_file   = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

# Prep the data

In [20]:
%%time

tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))
data = MovieReviewData(tokenizer, 
                       sample_size=1024,
                       max_seq_len=128)

neg: 100%|██████████| 12500/12500 [00:01<00:00, 8201.28it/s]
100%|██████████| 1.02k/1.02k [00:03<00:00, 292it/s]
100%|██████████| 1.02k/1.02k [00:03<00:00, 283it/s]


max seq_len 2900
CPU times: user 24.9 s, sys: 9.06 s, total: 34 s
Wall time: 48.5 s


# Let's take a look at the data

before...

In [21]:
data.train.head(2)

,sentence,sentiment,polarity
9660,"I had high hopes for this film, since it has Charlton Heston and Jack Palance. But those hopes came crashing to earth in the first 20 minutes or so. Palance was ridiculous. Not even Heston's acting or Annabel Schofield's beauty (or brief nude scenes) could save this film. Some of the space effects were quite good, but others were cheesy. The plot was ludicrous. Even sci-fi fans should skip this one. Grade F",3,0
23516,"I caught this film at the Edinburgh Film Festival. I hadn't heard much about it; only that it was a tightly-paced thriller, shot digitally on a very low budget. I was hoping to catch the next big Brit-Flick. But I have to say, I was severely disappointed. ""This Is Not A Love Song"" follows two criminals, who, after accidentally shooting and killing a farmer's young daughter, become embroiled in a deadly game of cat and mouse when the locals decide to take matters into their own hands and hunt them down.<br /><br />The real problem is that this is yet another example of style over substance in a British film. The camera angles and editing are completely at odds with the story, as are the over the top performances, and the appalling use of slow motion, which only serves to make the whole thing look like an expensive home video. There are repeated attempts to make the film look edgy and gritty, which instead come over as hilarious and over the top(Cue a pathetic, obligatory drug scene, and countless, pointless camera zooms). No amount of cliche's such as this can disguise the fact that this is a pretty bad story.<br /><br />We've seen this kind of thing many times before, and made a hundred times better, particularly in John Boorman's masterful ""Deliverance."" But while in the latter film, we actually cared about the characters, in this film, I found myself just wanting them to be hunted down and killed as quickly as possible. Even this wouldn't have been so bad if their adversaries had been frightening or worthwhile, but instead, are merely a collection of stereotypical, inbred-looking countryfolk. Again, another offensive, overused cliche' coming to the fore. Surely there are some nice people in the country, filmmakers?<br /><br />In its defense, ""This Is Not A Love Song"" does contain a couple of good, suspenseful moments, but it's hard to see this film doing anything other than going straight to video, or, at a push, getting a very limited cinema release. It's not a patch on last year's Low-Budget hunted in the hills movie, ""Dog soldiers"". Maybe British Cinema could actually get kick-started again if the right money stopped going to the wrong people.",1,0


after...

In [22]:
print("train_x:\n", data.train_x)
print("train_x shape:", data.train_x.shape)
print("train_x_token_types:\n", data.train_x_token_types)
print("train_x_token_types shape:", data.train_x_token_types.shape)
print("train_y:", data.train_y)
print("train_y shape:", data.train_y.shape)
print("max_seq_len:", data.max_seq_len)

train_x:
 [[  101  1045  2018 ...     0     0     0]
 [  101  1045  3236 ...  3291     0     0]
 [  101  1045  2245 ...  3531     0     0]
 ...
 [  101  1037 10103 ...  1028     0     0]
 [  101  2023  2003 ...  1045     0     0]
 [  101  1996  2197 ...  2250     0     0]]
train_x shape: (1024, 128)
train_x_token_types:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
train_x_token_types shape: (1024, 128)
train_y: [0 0 1 ... 0 1 1]
train_y shape: (1024,)
max_seq_len: 128


# Create Model

Initialize BERT

In [0]:
with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      bc = StockBertConfig.from_json_string(reader.read())
      bert_params = map_stock_config_to_params(bc)
      bert = BertModelLayer.from_params(bert_params, name="bert")

Build model architecture

In [0]:
input_ids = keras.layers.Input(shape=(data.max_seq_len,), dtype='int32', name="input_ids")
output = bert(input_ids)

cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(output)
cls_out = keras.layers.Dropout(0.5)(cls_out)
logits = keras.layers.Dense(units=768, activation="relu")(cls_out)
logits = keras.layers.Dropout(0.5)(logits)
logits = keras.layers.Dense(units=2, activation="softmax")(logits)

model = keras.Model(inputs=input_ids, outputs=logits)
model.build(input_shape=(None, data.max_seq_len))


Load BERT weights


In [25]:
load_stock_weights(bert, bert_ckpt_file)


Done loading 196 BERT weights from: .model/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7fa183607470> (prefix:bert_1). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


[]

Compile the model

In [0]:
model.compile(optimizer=keras.optimizers.Adam(),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

Let's take a look at the model

In [27]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 128)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 128, 768)          108890112 
_________________________________________________________________
lambda_1 (Lambda)            (None, 768)               0         
_________________________________________________________________
dropout_76 (Dropout)         (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 768)               590592    
_________________________________________________________________
dropout_77 (Dropout)         (None, 768)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1538

Define learning rate scheduler

In [0]:
def create_learning_rate_scheduler(max_learn_rate=5e-5,
                                   end_learn_rate=1e-7,
                                   warmup_epoch_count=10,
                                   total_epoch_count=90):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = (max_learn_rate/warmup_epoch_count) * (epoch + 1)
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler

# Train Model

In [29]:
%%time

log_dir = ".log/movie_reviews/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

total_epoch_count = 50
# model.fit(x=(data.train_x, data.train_x_token_types), y=data.train_y,
model.fit(x=data.train_x, y=data.train_y,
          validation_split=0.1,
          batch_size=48,
          shuffle=True,
          epochs=total_epoch_count,
          callbacks=[create_learning_rate_scheduler(max_learn_rate=1e-5,
                                                    end_learn_rate=1e-7,
                                                    warmup_epoch_count=20,
                                                    total_epoch_count=total_epoch_count),
                     keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True),
                     tensorboard_callback])

model.save_weights('./movie_reviews.h5', overwrite=True)

Train on 921 samples, validate on 103 samples

Epoch 00001: LearningRateScheduler reducing learning rate to 5.000000000000001e-07.
Epoch 1/50
921/921 [==============================] - 22s 24ms/sample - loss: 0.7150 - acc: 0.4832 - val_loss: 0.7133 - val_acc: 0.4563

Epoch 00002: LearningRateScheduler reducing learning rate to 1.0000000000000002e-06.
Epoch 2/50
921/921 [==============================] - 14s 16ms/sample - loss: 0.7257 - acc: 0.4886 - val_loss: 0.7028 - val_acc: 0.4660

Epoch 00003: LearningRateScheduler reducing learning rate to 1.5000000000000002e-06.
Epoch 3/50
921/921 [==============================] - 14s 16ms/sample - loss: 0.7144 - acc: 0.5005 - val_loss: 0.6924 - val_acc: 0.5049

Epoch 00004: LearningRateScheduler reducing learning rate to 2.0000000000000003e-06.
Epoch 4/50
921/921 [==============================] - 14s 16ms/sample - loss: 0.7035 - acc: 0.5353 - val_loss: 0.6832 - val_acc: 0.5437

Epoch 00005: LearningRateScheduler reducing learning rate to 2.500

# Evaluation

In [32]:
%%time 

# model = create_model(data.max_seq_len, adapter_size=None)
# model.load_weights("movie_reviews.h5")

_, train_acc = model.evaluate(data.train_x, data.train_y)
_, test_acc = model.evaluate(data.test_x, data.test_y)

print("train acc", train_acc)
print(" test acc", test_acc)

1024/1024 [==============================] - 6s 6ms/sample - loss: 0.4882 - acc: 0.8213
train acc 0.96875
 test acc 0.82128906
CPU times: user 3.47 s, sys: 697 ms, total: 4.16 s
Wall time: 11.4 s


# Prediction

In [34]:
pred_sentences = [
  "That movie was absolutely awful",
  "The movie was so boring",
  "The film was creative and surprising",
  "Absolutely fantastic!"
]

tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))
pred_tokens    = map(tokenizer.tokenize, pred_sentences)
pred_tokens    = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))

pred_token_ids = map(lambda tids: tids +[0]*(data.max_seq_len-len(tids)),pred_token_ids)
pred_token_ids = np.array(list(pred_token_ids))

print('pred_token_ids', pred_token_ids.shape)

res = model.predict(pred_token_ids).argmax(axis=-1)

for text, sentiment in zip(pred_sentences, res):
  print(" text:", text)
  print("  res:", ["negative","positive"][sentiment])

pred_token_ids (4, 128)
 text: That movie was absolutely awful
  res: negative
 text: The movie was so boring
  res: negative
 text: The film was creative and surprising
  res: positive
 text: Absolutely fantastic!
  res: positive
